In [107]:
# create a table and enable cdf
# create a sink table
# insert data source
# check history
# check cdf
# read feed stream
# apply foreachbatch function to apply changes


StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 109, Finished, Available)

In [108]:
%%sql

CREATE TABLE names (id INT, name STRING, age INT)

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 110, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [109]:
%%sql

CREATE TABLE names_sink (id INT, name STRING, age INT, age_days INT) 

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 111, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [110]:
%%sql 

ALTER TABLE names SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 112, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [111]:
from delta.tables import *

delta_names = DeltaTable.forName(spark, 'names')

fullHistoryDF = delta_names.history()

display(fullHistoryDF)

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 113, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4ecdd63a-2ffa-48c6-86c9-893055f6d7ea)

In [112]:
%%sql

insert into names (id, name, age)
values (1, 'Pedro', 47)

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 114, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [113]:
cdf_batch = spark.read.format("delta") \
  .option("readChangeFeed", "true") \
  .option("startingVersion", 1) \
  .table("names")
display(cdf_batch)

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 115, Finished, Available)

SynapseWidget(Synapse.DataFrame, b8f053b4-2613-488f-8346-cdfda0e90b9d)

In [114]:
from pyspark.sql import functions as F
def transform(df):
    df = df.withColumn('age_days', (F.col('age')*365))
    return df

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 116, Finished, Available)

In [115]:
names = spark.read.table('names')
display(transform(names))

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 117, Finished, Available)

SynapseWidget(Synapse.DataFrame, f390118d-5f76-43d4-b6df-455517637f28)

In [95]:
from pyspark.sql.window import Window
def cdf_id_last_version(df):
    filtered_df = (df 
        .filter(~F.col("_change_type").isin("delete", "update_preimage")) 
    )

    windowPartition = Window.partitionBy("id").orderBy(F.desc("_commit_timestamp"))

    ranked_df = (filtered_df 
        .withColumn("rnk", F.row_number().over(windowPartition))
    )

    result_df = ranked_df.filter(F.col("rnk") == 1)

    return result_df

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 97, Finished, Available)

In [116]:
def _batch_processing(df, batch_id):
    t_df = transform(df)
    t_df = cdf_id_last_version(t_df)
    sink_delta = DeltaTable.forName(spark, 'names_sink')
    sink_delta.alias('target') \
    .merge(t_df.alias('source'), 'source.id = target.id') \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 118, Finished, Available)

In [126]:
SOURCE_TABLE = "names"
checkpoint_path = "Files/_checkpoint/names"

strm = spark.readStream \
  .format("delta") \
  .option("readChangeFeed", "true") \
  .table(SOURCE_TABLE) \
  .writeStream \
  .foreachBatch(_batch_processing) \
  .trigger(availableNow=True) \
  .option("checkpointLocation", checkpoint_path) \
  .start()

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 128, Finished, Available)

In [127]:
strm.status

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 129, Finished, Available)

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}

In [119]:
%%sql

insert into names (id, name, age)
values (2, 'Lilli', 10)

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 121, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [123]:
%%sql

UPDATE names
set age = 11
where id =2

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 125, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [128]:
%%sql

select * from names_sink

StatementMeta(, 579d4a5f-8a01-4bc2-9ff8-ac37001c9887, 130, Finished, Available)

<Spark SQL result set with 2 rows and 4 fields>